# 📘 Spectroscopy Peaks: Lorentzian Line Shape Analysis> Fit Lorentzian profiles to spectroscopic peaks⏱️ **20-25 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Physics** | 🔬 **Optics**---

## 🔬 Domain Background**Model:** $I(\nu) = I_0 \frac{(\Gamma/2)^2}{(\nu - \nu_0)^2 + (\Gamma/2)^2}$**Where:**- $I_0$: Peak intensity- $\nu_0$: Center frequency- $\Gamma$: Full width at half maximum (FWHM)**Applications:** Atomic spectroscopy, laser physics, NMR---

## Setup

In [ ]:
import numpy as npimport jax.numpy as jnpfrom nlsq import curve_fitimport matplotlib.pyplot as pltdef lorentzian(nu, I0, nu0, gamma):    return I0 * (gamma/2)**2 / ((nu - nu0)**2 + (gamma/2)**2)

## Generate Spectral Data

In [ ]:
I0_true, nu0_true, gamma_true = 100, 500, 5  # THznu = np.linspace(480, 520, 100)I_true = lorentzian(nu, I0_true, nu0_true, gamma_true)I = I_true + np.random.normal(0, 2, len(nu))plt.plot(nu, I, 'o', alpha=0.3, label='Measured spectrum')plt.plot(nu, I_true, 'r--', label='True peak')plt.xlabel('Frequency (THz)')plt.ylabel('Intensity')plt.legend()plt.show()

## Fit Lorentzian Profile

In [ ]:
popt, pcov = curve_fit(lorentzian, nu, I, p0=[95, 499, 4])I0_fit, nu0_fit, gamma_fit = poptperr = np.sqrt(np.diag(pcov))print(f'Peak Analysis:')print(f'  Center frequency (ν₀): {nu0_fit:.2f} ± {perr[1]:.2f} THz')print(f'  Peak intensity (I₀):   {I0_fit:.1f} ± {perr[0]:.1f}')print(f'  FWHM (Γ):              {gamma_fit:.2f} ± {perr[2]:.2f} THz')print(f'\nQuality factor Q = ν₀/Γ = {nu0_fit/gamma_fit:.1f}')

## Visualize Fit

In [ ]:
I_fit = lorentzian(nu, *popt)plt.figure(figsize=(10,4))plt.subplot(121)plt.plot(nu, I, 'o', alpha=0.3, label='Data')plt.plot(nu, I_fit, 'g-', linewidth=2, label='Fit')plt.axvline(nu0_fit, color='orange', linestyle='--', alpha=0.5)plt.axhline(I0_fit/2, color='orange', linestyle='--', alpha=0.5)plt.xlabel('Frequency (THz)')plt.ylabel('Intensity')plt.legend()plt.subplot(122)residuals = I - I_fitplt.plot(nu, residuals, '.')plt.axhline(0, color='r', linestyle='--')plt.xlabel('Frequency (THz)')plt.ylabel('Residuals')plt.tight_layout()plt.show()

## Key Insights1. **Lorentzian shape** from natural broadening2. **FWHM** indicates spectral line width3. **Quality factor Q** measures line sharpness---